In [60]:
### Generate dataset from MT5

In [61]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime

# Connect to MetaTrader 5
if not mt5.initialize():
    print("MT5 Initialization failed:", mt5.last_error())
    quit()

# Set symbol
symbol = "XAUUSD"

# Ensure symbol is available
if not mt5.symbol_select(symbol, True):
    print(f"Failed to select {symbol}")
    mt5.shutdown()
    quit()

# Get last 1000 1-minute candles
rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, 0, 1000)

# Convert to DataFrame
df = pd.DataFrame(rates)
df['time'] = pd.to_datetime(df['time'], unit='s')

# Show last few rows
print(df.tail())

# Save for training
df.to_csv("xauusd_data.csv", index=False)

# Disconnect
mt5.shutdown()


                   time     open     high      low    close  tick_volume  \
995 2025-07-02 09:35:00  3340.99  3341.76  3339.10  3339.10          201   
996 2025-07-02 09:36:00  3339.10  3339.42  3338.60  3339.04          191   
997 2025-07-02 09:37:00  3339.04  3341.22  3338.56  3340.77          187   
998 2025-07-02 09:38:00  3340.77  3341.08  3340.51  3340.51          115   
999 2025-07-02 09:39:00  3340.51  3341.23  3340.49  3341.23           94   

     spread  real_volume  
995      12            0  
996      12            0  
997      12            0  
998      12            0  
999      12            0  


True

In [62]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
import joblib
import os

In [63]:
# === Step 1: Load and explore the dataset
df = pd.read_csv("xauusd_data.csv")

In [64]:
df.head()

,time,open,high,low,close,tick_volume,spread,real_volume
0,2025-07-01 15:58:00,3340.29,3340.31,3339.23,3339.27,126,12,0
1,2025-07-01 15:59:00,3339.29,3340.28,3339.29,3340.27,111,12,0
2,2025-07-01 16:00:00,3340.28,3340.96,3339.80,3340.74,167,12,0
3,2025-07-01 16:01:00,3340.79,3342.03,3340.77,3341.87,157,12,0
4,2025-07-01 16:02:00,3341.87,3341.95,3341.36,3341.61,114,12,0


In [65]:
# Print the last 60 closing prices as comma-separated integers
print(",".join([str(int(val)) for val in df["close"].tail(60)]))

3336,3335,3335,3335,3336,3337,3335,3336,3336,3335,3334,3334,3334,3333,3334,3334,3333,3333,3333,3333,3333,3333,3334,3334,3334,3334,3334,3334,3335,3335,3335,3335,3336,3335,3336,3337,3338,3338,3338,3337,3338,3338,3338,3338,3338,3338,3338,3338,3339,3339,3338,3338,3340,3340,3341,3339,3339,3340,3340,3341


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   time         1000 non-null   object 
 1   open         1000 non-null   float64
 2   high         1000 non-null   float64
 3   low          1000 non-null   float64
 4   close        1000 non-null   float64
 5   tick_volume  1000 non-null   int64  
 6   spread       1000 non-null   int64  
 7   real_volume  1000 non-null   int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 62.6+ KB


In [67]:
# === Step 1: Load and clean the dataset ===
df = pd.read_csv("xauusd_data.csv")
df['time'] = pd.to_datetime(df['time'])

In [68]:
# Focus on 'close' column
close_data = df['close'].values.reshape(-1, 1)

In [69]:
# === Step 2: Normalize the data ===
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(close_data)

In [70]:
# === Step 3: Create 60-step sequences for XGBoost ===
def create_sequences(data, window=60):
    X, y = [], []
    for i in range(window, len(data)):
        X.append(data[i-window:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, window=60)

In [71]:
# === Step 4: Train-test split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [72]:
# === Step 5: Train XGBoost model ===
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
xgb_model.fit(X_train, y_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [73]:
# === Step 6: Save model + scaler ===
os.makedirs("models", exist_ok=True)
xgb_model.save_model("models/xgboost_model.json")
joblib.dump(scaler, "models/scaler.pkl")

print("✅ XGBoost model and scaler saved in /models")

✅ XGBoost model and scaler saved in /models


In [74]:
from fastapi import FastAPI, Request, Query
from fastapi.templating import Jinja2Templates
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from app.forecast_logic import forecast_next
from pydantic import BaseModel
from typing import List, Optional

app = FastAPI(
    title="XAUUSD Forecast API",
    description="AI-powered forecast and trading signal generator for gold (XAUUSD) using XGBoost.",
    version="1.0.0"
)

# HTML template directory
templates = Jinja2Templates(directory="templates")

# CORS middleware setup (Allow all origins for dev; restrict in production)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Change to ["https://yourdomain.com"] in production
    allow_methods=["*"],
    allow_headers=["*"]
)

# Input schema for closing prices
class PriceInput(BaseModel):
    close_prices: List[float]

# Root route for rendering frontend UI
@app.get("/", response_class=HTMLResponse)
def read_root(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

# Health check route
@app.get("/ping")
def ping():
    return {"status": "ok"}

# Prediction route with optional threshold
@app.post("/predict")
def predict(input: PriceInput, threshold: Optional[float] = Query(0.002, ge=0, le=1)):
    """
    Forecast next gold price and trading signal.
    Accepts last 60 close prices and an optional threshold for signal sensitivity.
    """
    try:
        result = forecast_next(input.close_prices, threshold)
        return JSONResponse(content=result)
    except ValueError as ve:
        return JSONResponse(content={"error": str(ve)}, status_code=422)
    except Exception as e:
        return JSONResponse(content={"error": "Server error", "details": str(e)}, status_code=500)


In [75]:
import numpy as np
import joblib
import xgboost as xgb

# Load XGBoost model and scaler
xgb_model = xgb.XGBRegressor()
xgb_model.load_model("models/xgboost_model.json")
scaler = joblib.load("models/scaler.pkl")

# Signal generator with adjustable threshold
def generate_signal(predicted_price, current_price, threshold=0.002):
    diff_ratio = (predicted_price - current_price) / current_price
    if diff_ratio > threshold:
        return "BUY"
    elif diff_ratio < -threshold:
        return "SELL"
    else:
        return "HOLD"

# Main forecast function
def forecast_next(close_prices: list, threshold: float = 0.002):
    if len(close_prices) < 60:
        raise ValueError("Input must have at least 60 closing prices.")

    # Prepare the last 60 prices
    recent_data = np.array(close_prices[-60:]).reshape(-1, 1)

    # Scale and reshape
    scaled = scaler.transform(recent_data)
    X_xgb = scaled.reshape(1, 60)

    # Predict next scaled value
    xgb_pred = xgb_model.predict(X_xgb)
    predicted_scaled = xgb_pred[0]

    # Inverse scale to get real price
    predicted_price = scaler.inverse_transform([[predicted_scaled]])[0][0]

    # Extract current price
    current_price = close_prices[-1]

    # Generate buy/sell/hold signal
    signal = generate_signal(predicted_price, current_price, threshold)

    # TP & SL logic (buffer = 0.5%)
    buffer = 0.005
    if signal == "BUY":
        tp = predicted_price * (1 + buffer)
        sl = predicted_price * (1 - buffer)
    elif signal == "SELL":
        tp = predicted_price * (1 - buffer)
        sl = predicted_price * (1 + buffer)
    else:  # HOLD
        tp = sl = predicted_price

    # Final response
    return {
        "predicted_price": round(predicted_price, 3),
        "current_price": round(current_price, 3),
        "signal": signal,
        "take_profit": round(tp, 3),
        "stop_loss": round(sl, 3)
    }


In [76]:
fastapi==0.110.0
uvicorn==0.29.0
jinja2==3.1.3
numpy==1.24.4
scikit-learn==1.3.0
xgboost==1.7.6
joblib==1.3.2
setuptools>=42
wheel

SyntaxError: invalid syntax (1012526206.py, line 1)